# Analysis of RNA-Seq Data

Piero Palacios Bernuy [](https://orcid.org/0000-0001-6729-4080)  
February 28, 2024

This document is part of a series of the analysis of Omics data. Especifycally, here is showed how to analyze bulk RNA-Seq data with Bioconductor packages. Also, it’s showcased how to make plots of the rna data in the context of differentially gene expression and gene-sets.

## 1 Introduction

hola

## 2 Data & Methods

In [ ]:
library(airway)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: 'MatrixGenerics'

The following objects are masked from 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges, rowRanks,
    rowSdDiffs, rowSds, rowSums2, rowTabulates, rowVarDiffs, rowVars,
    rowWeightedMads, rowWeightedMeans, rowWeightedMedians,
    rowWeightedSds, rowWeig

...

Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Attaching package: 'Biobase'

The following object is masked from 'package:MatrixGenerics':

    rowMedians

The following objects are masked from 'package:matrixStats':

    anyMissing, rowMedians

pre-filtering

In [ ]:
keep <- rowSums(counts(dds)>= 10) >= 3

dds <- dds[keep,]

re-leveling

In [ ]:
dds$dex <- factor(dds$dex, levels = c("untrt","trt"))
dds$dex <- relevel(dds$dex, ref = "untrt")
dds$dex <- droplevels(dds$dex)

differential gene expression

In [ ]:
dds <- DESeq(dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

log2 fold change (MLE): dex trt vs untrt 
Wald test p-value: dex trt vs untrt 
DataFrame with 16596 rows and 6 columns
                 baseMean log2FoldChange     lfcSE      stat      pvalue
                <numeric>      <numeric> <numeric> <numeric>   <numeric>
ENSG00000000003   709.880     -0.3839261 0.1008515 -3.806846 1.40750e-04
ENSG00000000419   521.156      0.2041705 0.1116546  1.828590 6.74611e-02
ENSG00000000457   237.573      0.0352858 0.1412422  0.249825 8.02723e-01
ENSG00000000460    58.035     -0.0923157 0.2792981 -0.330527 7.41002e-01
ENSG00000000971  5826.538      0.4237405 0.0893546  4.742236 2.11372e-06
...                   ...            ...       ...       ...         ...
ENSG00000273448  14.02944      0.0583225  0.483593  0.120603    0.904006
ENSG00000273472  11.08483     -0.4087635  0.558196 -0.732294    0.463989
ENSG00000273486  15.47814     -0.1516640  0.471121 -0.321922    0.747512
ENSG00000273487   8.17605      1.0408751  0.678127  1.534926    0.124802
ENSG0

## 3 Conclusion

hola

## References